In [17]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom
import nibabel.nicom.csareader as csareader
import os
import pandas as pd

In [18]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

# directories will have to be specific to your computer
base_dir = '/Users/amattfel/Documents/GitHub'
proj_dir = '/mattfeld_2020/sourcedata'
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/6-fMRI_DistortionMap_PA/resources/DICOM/files'

# with strings I can concatenate them with simple addition lines
data_dir = base_dir + proj_dir + subj_dir

# Here I use glob to grab the dicom files
# Why would I use glob?  What does it give me?
data_files = glob(data_dir + '/*')


In [19]:
data_files

[]

In [20]:
# Here I am using the python dicom reader to read in a dicom header
# What should go between the square brackets []
data_set = dicom.dcmread(data_files[0])

IndexError: list index out of range

In [21]:
data_set

NameError: name 'data_set' is not defined

In [22]:
# Elements we're interested in
# Repetition Time
# Echo Time
# Acquisition Matrix 
# Flip Angle
# Acquisition Number
# Protocol Name
# Slice times = data_set[0x00191029].value

print('The number of TRs collected equals: {0}'.format(len(data_files)))
print('The timing of the TR was: {0} ms'.format(data_set.RepetitionTime))
print('The timing of the TE was: {0} ms'.format(data_set.EchoTime))
print('The flip angle in degrees was: {0}'.format(data_set.FlipAngle))
print('The name of the scan was: {0}'.format(data_set.FlipAngle))
print('The phase encode direction was: {0}'.format(data_set[int('00181312', 16)].value))
print('COL = AP or PA; ROW = RL or LR')
#print('The slice timing was: {0}'.format(data_set[0x00191029].value))
print('The field of view - FOV was: {0}'.format(data_set[int('0051100c', 16)].value))

# Why use the following format to access information in the header?
#print(data_set[int('0051100c', 16)].value) # data_set[0x0051100c].value
# Allows us to access information that is private in the dicom header.

# How do I find phase encoding direction information?

csa_str = data_set[int('00291010', 16)].value
csa_tr = csareader.read(csa_str)
print('The phase encode direction was: {0}'.format(csa_tr['tags']['PhaseEncodingDirectionPositive']['items'][0]))
print('1 = +; 0 = -')
print('COL, 1 = PA; COL, 0 = AP')



The number of TRs collected equals: 0


NameError: name 'data_set' is not defined

# Using heudiconv to convert dicoms into BIDs format

In [15]:
# Step 1 - run heudiconv
#heudiconv -d '../sourcedata/Mattfeld_REVL-000-vCAT-{subject}-S1/*/*/*/*/*/*' -s 021 -c none -f convertall -o /Users/amattfel/Documents/GitHub/mattfeld_2020/dset

#/Users/amattfel/Documents/GitHub/mattfeld_2020/dset/.heudiconv/021/info
path2info_file = os.path.join(base_dir, 'mattfeld_2020', 'dset', '.heudiconv/021/info')
dicominfo_df = pd.read_csv(path2info_file + '/dicominfo.tsv', sep="\t")
limited_dicominfo_df = dicominfo_df[['series_id', 'series_files', 'dim1', 'dim2', 'dim3', 'dim4', 'series_description']]

limited_dicominfo_df


FileNotFoundError: [Errno 2] No such file or directory: '/Users/amattfel/Documents/GitHub/mattfeld_2020/dset/.heudiconv/021/info/dicominfo.tsv'

heudiconv -d "/Users/amattfel/Documents/GitHub/mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-{subject}-S1/*/*/*/D*/*/*" -s 021 -f psb6351_heuristic.py -b -o /Users/amattfel/Documents/GitHub/mattfeld_2020/dset

In [16]:
dicominfo_df.keys()

NameError: name 'dicominfo_df' is not defined